In [1]:
import os
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from pydicom import dcmread
import shutil
import subprocess as sp

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

In [2]:
df = pd.read_csv('../datasets/external/vindr/all_labels.csv')
df

,patient_id,series_id,image_id,laterality,view,age,BIRADS,density,split,__SmallestImagePixelValue,__PixelRepresentation,__SOPInstanceUID,__PatientWeight,__WindowWidth,__PixelPaddingValue,__BodyPartExamined,__StudyInstanceUID,__ViewPosition,__LargestImagePixelValue,__PixelSpacing,__PixelPaddingRangeLimit,__SeriesInstanceUID,__SamplesPerPixel,__PresentationLUTShape,__WindowCenterWidthExplanation,__WindowCenter,__ImagerPixelSpacing,__VOILUTFunction,__RescaleIntercept,__RescaleSlope,__PhotometricInterpretation,__Rows,__Columns,__hasVOILUTSequence,__hasLUTDescriptor,__hasLUTData,cancer
0,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,d8125545210c08e1b1793a5af6458ee2,L,CC,053Y,2,C,training,NaN,0,d8125545210c08e1b1793a5af6458ee2,NaN,1500,0,BREAST,b8d273e8601f348d3664778dae0e7e0b,CC,NaN,NaN,NaN,b36517b9cbbcfd286a7ae04f643af97a,1,IDENTITY,linear LUT,1662,"[0.085, 0.085]",NaN,0.0,1.0,MONOCHROME2,3518,2800,False,False,False,0
1,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,290c658f4e75a3f83ec78a847414297c,L,MLO,053Y,2,C,training,NaN,0,290c658f4e75a3f83ec78a847414297c,NaN,1500,0,BREAST,b8d273e8601f348d3664778dae0e7e0b,MLO,NaN,NaN,NaN,b36517b9cbbcfd286a7ae04f643af97a,1,IDENTITY,linear LUT,1664,"[0.085, 0.085]",NaN,0.0,1.0,MONOCHROME2,3518,2800,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19998,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,d443b9725e331b8b27589aa725597801,R,MLO,018Y,2,C,training,NaN,0,d443b9725e331b8b27589aa725597801,NaN,"[1778, 1694, 1524, 1863, 1778]",0,BREAST,b3c8969cd2accfa4dbb2aece1f7158ab,MLO,NaN,"[0.0828, 0.0828]",294.0,69d7f07ea04572dad5e5aa62fbcfc4b7,1,IDENTITY,"['CURRENT', 'STANDARD', 'CONTRAST', 'SMOOTH', ...","[8324, 8185, 8152, 8270, 8324]","[0.0828, 0.0828]",LINEAR,0.0,1.0,MONOCHROME2,3580,2686,True,True,True,0
19999,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,45c1239cc36b0e672f0072707fd05c6f,L,MLO,018Y,2,C,training,NaN,0,45c1239cc36b0e672f0072707fd05c6f,NaN,"[1845, 1758, 1582, 1933, 1845]",0,BREAST,b3c8969cd2accfa4dbb2aece1f7158ab,MLO,NaN,"[0.0828, 0.0828]",1437.0,69d7f07ea04572dad5e5aa62fbcfc4b7,1,IDENTITY,"['CURRENT', 'STANDARD', 'CONTRAST', 'SMOOTH', ...","[8300, 8155, 8120, 8243, 8300]","[0.0828, 0.0828]",LINEAR,0.0,1.0,MONOCHROME2,3580,2670,True,True,True,0


In [3]:
df.patient_id.nunique()

5000

In [4]:
df.image_id.nunique()

20000

In [5]:
df.cancer.sum()

226

In [6]:
len(df[df.cancer == 1])

226

In [7]:
226 / 20000

0.0113

In [10]:
temp = df.groupby('patient_id')['cancer'].mean()
temp

patient_id
0025a5dc99fd5c742026f0b2b030d3e9    0.0
0028fb2c7f0b3a5cb9a80cb0e1cdbb91    0.0
                                   ... 
ffe7a45f8390f242db3b843762a4a7aa    0.0
fff2339ea4b5d2f1792672ba7d52b318    0.0
Name: cancer, Length: 5000, dtype: float64

In [11]:
len(temp[temp > 0])

113

In [12]:
df = pd.read_csv('../datasets/external/miniddsm/label_v2.csv')
df

,patient_id,image_id,view,laterality,density,age,ddsm_ori_status,cancer
0,C_0029_1,C_0029_1@LEFT@CC,CC,L,C,66.0,Benign,0
1,C_0029_1,C_0029_1@LEFT@MLO,MLO,L,C,66.0,Benign,0
...,...,...,...,...,...,...,...,...
7806,D_4608_1,D_4608_1@RIGHT@CC,CC,R,B,39.0,Normal,0
7807,D_4608_1,D_4608_1@RIGHT@MLO,MLO,R,B,39.0,Normal,0


In [13]:
df.patient_id.nunique()

1952

In [14]:
df.cancer.sum()

1480

In [15]:
1480 / 7808

0.18954918032786885

In [16]:
df = pd.read_csv('../datasets/external/cmmd/label_v2.csv')
df

,patient_id,image_id,laterality,age,cancer,_num_images,abnormality,classification,subtype,__VOILUTFunction,__WindowCenter,__WindowWidth,__WindowCenterWidthExplanation,__RescaleSlope,__RescaleIntercept,__Columns,__Rows,__SOPInstanceUID,__LossyImageCompression,__CodeMeaning,__StudyID,__PatientOrientation,__SpecificCharacterSet,__InstanceCreatorUID,__PositionerType,__PhotometricInterpretation,__BitsAllocated,__StudyDate,__PatientName,__ContentTime,__PixelIntensityRelationship,__Modality,__PresentationIntentType,__AcquisitionTime,__BodyPartExamined,__HighBit,__PatientIdentityRemoved,__BitsStored,__InstanceCreationTime,__StudyTime,__SeriesTime,__PatientBirthDate,__ImageType,__RescaleType,__SeriesInstanceUID,__ReferringPhysicianName,__ContentDate,__InstanceCreationDate,__PixelIntensityRelationshipSign,__SeriesNumber,__StudyInstanceUID,__PatientID,__DetectorType,__SamplesPerPixel,__SOPClassUID,__PixelRepresentation,__CodeValue,__OrganExposed,__InstanceNumber,__AccessionNumber,__SeriesDate,__AcquisitionDate,__PatientAge,__BurnedInAnnotation,__PresentationLUTShape,__Manufacturer,__DeidentificationMethod,__ImagerPixelSpacing,__PatientSex
0,D1-0830,D1-0830_1-1,L,63.0,0,2.0,mass,Benign,NaN,SIGMOID,128.0,256.0,Full width of 8 bit data,1.0,0.0,1914,2294,1.3.6.1.4.1.14519.5.2.1.1239.1759.244359496749...,0,cranio-caudal,NaN,"['A', 'R']",ISO_IR 100,1.3.6.1.4.1.14519.5.2.1.1239.1759.256693145395...,MAMMOGRAPHIC,MONOCHROME2,8,20100718,D1-0830,0,LOG,MG,FOR PRESENTATION,0,BREAST,7,YES,8,120428,0,0,NaN,"['DERIVED', 'PRIMARY']",US,1.3.6.1.4.1.14519.5.2.1.1239.1759.320712920134...,NaN,20100718,20170818,-1,1,1.3.6.1.4.1.14519.5.2.1.1239.1759.160257839898...,D1-0830,SCINTILLATOR,1,1.2.840.10008.5.1.4.1.1.1.2,0,399162004,BREAST,1,NaN,20100718,20100718,063Y,NO,IDENTITY,NaN,"Per DICOM PS 3.15 AnnexE. Details in 0012,0064","[0.094090909, 0.094090909]",F
1,D1-0830,D1-0830_1-2,L,63.0,0,2.0,mass,Benign,NaN,SIGMOID,128.0,256.0,Full width of 8 bit data,1.0,0.0,1914,2294,1.3.6.1.4.1.14519.5.2.1.1239.1759.320677583371...,0,medio-lateral oblique,NaN,"['A', 'FR']",ISO_IR 100,1.3.6.1.4.1.14519.5.2.1.1239.1759.256693145395...,MAMMOGRAPHIC,MONOCHROME2,8,20100718,D1-0830,0,LOG,MG,FOR PRESENTATION,0,BREAST,7,YES,8,120429,0,0,NaN,"['DERIVED', 'PRIMARY']",US,1.3.6.1.4.1.14519.5.2.1.1239.1759.320712920134...,NaN,20100718,20170818,-1,1,1.3.6.1.4.1.14519.5.2.1.1239.1759.160257839898...,D1-0830,SCINTILLATOR,1,1.2.840.10008.5.1.4.1.1.1.2,0,399368009,BREAST,2,NaN,20100718,20100718,063Y,NO,IDENTITY,NaN,"Per DICOM PS 3.15 AnnexE. Details in 0012,0064","[0.094090909, 0.094090909]",F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200,D2-0448,D2-0448_1-4,R,NaN,0,NaN,NaN,Normal,NaN,SIGMOID,128.0,256.0,Full width of 8 bit data,1.0,0.0,1914,2294,1.3.6.1.4.1.14519.5.2.1.1239.1759.704200754105...,0,medio-lateral oblique,NaN,"['P', 'FL']",ISO_IR 100,1.3.6.1.4.1.14519.5.2.1.1239.1759.256693145395...,MAMMOGRAPHIC,MONOCHROME2,8,20100718,D2-0448,0,LOG,MG,FOR PRESENTATION,0,BREAST,7,YES,8,124254,0,0,NaN,"['DERIVED', 'PRIMARY']",US,1.3.6.1.4.1.14519.5.2.1.1239.1759.378084371300...,NaN,20100718,20170818,-1,1,1.3.6.1.4.1.14519.5.2.1.1239.1759.744773683160...,D2-0448,SCINTILLATOR,1,1.2.840.10008.5.1.4.1.1.1.2,0,399368009,BREAST,4,NaN,20100718,20100718,053Y,NO,IDENTITY,NaN,"Per DICOM PS 3.15 AnnexE. Details in 0012,0064","[0.094090909, 0.094090909]",F
5201,D2-0448,D2-0448_1-3,R,NaN,0,NaN,NaN,Normal,NaN,SIGMOID,128.0,256.0,Full width of 8 bit data,1.0,0.0,1914,2294,1.3.6.1.4.1.14519.5.2.1.1239.1759.170184140124...,0,cranio-caudal,NaN,"['P', 'L']",ISO_IR 100,1.3.6.1.4.1.14519.5.2.1.1239.1759.256693145395...,MAMMOGRAPHIC,MONOCHROME2,8,20100718,D2-0448,0,LOG,MG,FOR PRESENTATION,0,BREAST,7,YES,8,124253,0,0,NaN,"['DERIVED', 'PRIMARY']",US,1.3.6.1.4.1.14519.5.2.1.1239.1759.37

In [17]:
df.patient_id.nunique()

1775

In [18]:
df.cancer.sum()

2632

In [19]:
2632 / 5202

0.5059592464436755

In [20]:
df = pd.read_csv('../datasets/external/cdd_cesm/label_v2.csv')
df

,patient_id,image_id,laterality,view,image_name,age,density,BIRADS,findings,tags,machine_id,classification,Type,cancer
0,1,P1_L_MLO,L,MLO,P1_L_DM_MLO,46,C,4,Architectural distortion,"suspicious, distortion",1,Malignant,DM,1
1,2,P2_R_CC,R,CC,P2_R_DM_CC,31,C,3$2,Multiple equal density masses$Benign macrocal...,"benign, masses, macrocalcification",2,Benign,DM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,326,P326_L_CC,L,CC,P326_L_DM_CC,51,C,3,Asymmetrical increased density,asymmetry,1,Malignant,DM,1
1002,326,P326_L_MLO,L,MLO,P326_L_DM_MLO,51,C,3,Asymmetrical increased density,asymmetry,1,Malignant,DM,1


In [21]:
df.patient_id.nunique()

326

In [22]:
df.cancer.sum()

331

In [23]:
331 / 1003

0.3300099700897308

In [24]:
df = pd.read_csv('../datasets/external/bmcd/label_v2.csv')
df

,patient_id,laterality,age,density,BIRADS,biopsy_result,folder_type,view,image_id,cancer,__WindowWidth,__WindowCenter,__WindowCenterWidthExplanation,__RescaleType,__RescaleSlope,__RescaleIntercept,__Rows,__Columns,__BitsAllocated,__BitsStored
0,suspicious_1,Right,57,b,4a,BENIGN,suspicious,MLO,suspicious_1MLO_recent,0,4096.0,2047.0,NaN,US,1.0,0.0,4096,3328,16,12
1,suspicious_1,Right,57,b,4a,BENIGN,suspicious,CC,suspicious_1CC_recent,0,4096.0,2047.0,NaN,US,1.0,0.0,4096,3328,16,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,normal_50,Right,56,b,2,NAN,normal,MLO,normal_50MLO_recent,0,4096.0,2047.0,NaN,US,1.0,0.0,4096,3328,16,12
327,normal_50,Right,56,b,2,NAN,normal,CC,normal_50CC_prior,0,4096.0,2047.0,NaN,US,1.0,0.0,4096,3328,16,12


In [25]:
df.patient_id.nunique()

82

In [26]:
df.cancer.sum()

22

In [27]:
22 / 328

0.06707317073170732

In [28]:
5000 + 1952 + 1775 + 326 + 82

9135

In [29]:
20000 + 7808 + 5202 + 1003 + 328


34341

In [30]:
226 + 1480 + 2632 + 331 + 22

4691

In [31]:
4691 / 34341

0.13660056492239597